In [2]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [3]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)     

In [4]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True)

In [5]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    # print(inputs)
    # print(labels)
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [6]:
labels

tensor([9, 4, 9, 8, 7, 2, 1, 7, 7, 2, 7, 9, 5, 3, 7, 9, 7, 3, 5, 0, 6, 2, 7, 4,
        8, 8, 9, 6, 2, 5, 7, 6, 5, 7, 0, 1, 1, 1, 5, 8, 4, 5, 6, 5, 4, 0, 8, 3,
        9, 3, 8, 3, 8, 4, 5, 8, 8, 0, 7, 9, 5, 2, 4, 2])

In [7]:
len(train_loader)

938

In [8]:
# 定义网络结构
# input_size输入特征的大小
# hidden_size, LSTM模块的数量
# num_layer, LSTM的层数
# LSTM默认input(seq_len, batch, feature)
# batch_first=True,input和output(seq_len, batch, feature)
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        self.out = torch.nn.Linear(in_features=64, out_features=10)
        self.softmax = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        # (batch, seq_len, feature)
        x = x.view(-1,28,28)
        # output:[batch, seq_len, hidden_size]包含每个序列的输出结果
        # 虽然LSTM的batch_first为True, 但是h_n, c_n的第一个维度还是num_layers
        # h_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        # c_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        output, (h_n, c_n)=self.lstm(x)
        output_in_last_timestep = h_n[-1,:,:]
        x = self.out(output_in_last_timestep)
        x = self.softmax(x)
        return x

In [9]:
LR = 0.0003
# 定义模型
model = LSTM()
# 定义损失函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

In [10]:
def train():
    # 训练状态
    model.train()
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构 (64,10)
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()
        
def test():
    # 测试状态
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(train_dataset)))

In [ ]:
for epoch in range(10):
    print('epoch:', epoch)
    train()
    test()

In [ ]:
# 模型保存
torch.save(model.state_dict(), 'model/my_model.pth')

In [ ]:
# 模型载入
# 载入训练好的模型参数
model.load_state_dict(torch.load('model/my_model.pth'))